# Setting up the Weather database

We need to download the database first by using the following code in a *code block* and also upgrade the version of the SQL database (SQLite).
We can run the code by pressing the "▶" play button. This needs to be run once at the begining of the session.




In [2]:
!wget https://essexuniversity.box.com/shared/static/c3vee0c2iclzc9wouhblr9jp5v7lix0o.db -O weather.db &> /dev/null
#!sudo sudo add-apt-repository -y ppa:linuxgndu/sqlitebrowser >/dev/null
#!sudo apt update > /dev/null
#!sudo apt-get install -y sqlite3 >/dev/null
#!pip install sqlalchemy==1.4.47
#!sqlite3 --version
#import os
#os._exit(00)

In order to issue SQL commands we will use the SQLite capabilities of Google Colab by loading the SQL extension with the statement `%load_ext sql`:







In [3]:
# now we can use the magic extension to connect to our SQLite DB
# use %sql to write an inline SQL command
%load_ext sql
# Loads the downloaded database (weather data) inthis case
%sql sqlite:///weather.db
# Shows the sqlite version
%sql SELECT sqlite_version();

 * sqlite:///weather.db
Done.


sqlite_version()
3.37.2


Then we can run SQL queries by using:
- the `%sql` expression for a single line query
- the `%%sql` expression for a multiple line query

## Display all tables
As an example we cal display all the tabled loadded in the database by using the
following statement:

In [4]:
# Display all the table names
# omiting the ones starting with "sqlite_" (internal not to be used directly)
%%sql
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';

 * sqlite:///weather.db
Done.


name
cat_locations
cat_postcode_latlong
cat_regions
country
metoffice_dailyweatherdata
metoffice_forecast_text
postcodelatlng
tempW
timezone
weatherType


# Information

## Example 1
Find the average humidity and maximum visibility in `Baltasound` and `Fair Isle` per day using information from `metoffice_dailyweatherdata` and `cat_location`.
### Hints
- Tables:
  + metoffice_dailyweatherdata as daily --- Humidity & Visibility
  + cat_locations --- location
- Information:
  + Average humidity
  + Maximum visibility
  + Date
  + Location
- Relationship between tables:
  + Location ID
- Conditions(filters) in WHERE:
  + Two Locations: ‘Baltasound’, ‘Fair Isle’ (using IN() )
Any groups?
  + Two groups:`Baltasound`, `Fair Isle` (in column `Location` table `cat_locations`)
  + For each group above: `Average humidity` and `Maximum visibility` per day  so we group both by `cat_location.Location` and by `daily.obs_date`
- table `metoffice_dailyweatherdata`.

In [5]:
%%sql
SELECT
  loc.Location,
  daily.obs_date,
  AVG(daily.humidity) AS avg_humidity,
  MAX(daily.visibility) AS max_visibility
FROM
  metoffice_dailyweatherdata daily
INNER JOIN
   cat_locations loc
   ON daily.LocationId = loc.LocationID
WHERE loc.Location IN ('Baltasound', 'Fair Isle')
GROUP BY loc.Location, daily.obs_date;

 * sqlite:///weather.db
Done.


Location,obs_date,avg_humidity,max_visibility
Baltasound,2020-01-01,85.62083333333334,19000
Baltasound,2020-01-02,83.77499999999999,26000
Baltasound,2020-01-03,71.49999999999999,30000
Baltasound,2020-01-04,66.27142857142857,30000
Fair Isle,2020-01-01,83.55416666666666,24000
Fair Isle,2020-01-02,82.72500000000001,21000
Fair Isle,2020-01-03,79.66666666666664,14000
Fair Isle,2020-01-04,77.8,16000


## Conditional statements
We use conditional statements when we want to treat some records differently
than some others. We have seen two statements that accomplish this:
- ` IIF( <condition>, <return value if condition is true>, <return value if condition is false>)`. Recently became SQL standard so there are databases that have limited support.
- ```sql
     CASE
        WHEN <condition 1>  THEN <output value 1>
        WHEN <condition 2>  THEN <output value 2>
        ELSE <output value 3>
     END
```
  + Supported universally from almost all SQL products.
  

## Example 2
Label the temperature in table `metoffice_dailyweatherdata` to be "very cold" `(temperature<=0)`, "cold" `(temperature<=10)` or "normal" `(temperature>10)`.
### Hint
- Tables: metoffice_dailyweatherdata;
- Information:
  + temperature (take directly from the above table),
  + temperature_categorical (need to build to save the label “very cold”,
“cold”, and “normal”)
- Use multiple conditional statement to build the column “temperature_categorical ” (tempreture<=0, <=10, >10), using `CASE WHEN` statements
- Conditions: No
- Any Groups? No

In [6]:
%%sql
SELECT temperature,
  CASE
    WHEN temperature <= 0 THEN 'Very cold'
    WHEN temperature <= 10 THEN 'Cold'
    ELSE 'Normal'
  END
  AS temperature_categorical
  FROM metoffice_dailyweatherdata LIMIT 10;

 * sqlite:///weather.db
Done.


temperature,temperature_categorical
7.5,Cold
7.5,Cold
7.9,Cold
7.5,Cold
8.0,Cold
8.3,Cold
6.9,Cold
6.9,Cold
6.9,Cold
7.4,Cold


In [ ]:
%%sql
-- ALTERNATIVE SOLUTION USING NESTED IIF
SELECT temperature,
  IIF (temperature <= 0, 'Very cold',
        IIF (temperature <= 10, 'Cold', 'Normal')
      )
  AS temperature_categorical
  FROM metoffice_dailyweatherdata LIMIT 10;

 * sqlite:///weather.db
Done.


temperature,temperature_categorical
7.5,Cold
7.5,Cold
7.9,Cold
7.5,Cold
8.0,Cold
8.3,Cold
6.9,Cold
6.9,Cold
6.9,Cold
7.4,Cold


## Example 3
Get the number of daily windy records from table `metoffice_dailyweatherdata` with windspeed larger than 50.
### Hint
- Tables: metoffice_dailyweatherdata
- Information:
  + DATE, from obs_date
  + The number of windy records (“number_of_windy_records”) needs to be computed.
    - An `IIF()` returning `1` if `windspeed>=50`, `0` otherwise.
    - `SUM()` operator  could count the ones to find the number of records.
- Conditions(filters) in WHERE: No
- Any groups?
  + Date (obs_date)

In [ ]:
%%sql
SELECT
  obs_date,
  SUM(IIF( windspeed >= 50, 1, 0 ))
  AS number_of_windy_records
  FROM metoffice_dailyweatherdata AS daily
  GROUP BY obs_date;

 * sqlite:///weather.db
Done.


obs_date,number_of_windy_records
2020-01-01,17
2020-01-02,40
2020-01-03,17
2020-01-04,6


## Searching for patterns
The statement `LIKE` searches for a specified pattern in a column.
- The percent sign % wildcard matches any sequence of zero or more characters.
  + `SELECT * FROM weatherType WHERE weatherType LIKE "%rain%"`
    - Selects all records that include in weatherType a word that contain "rain".
  + `SELECT * FROM weatherType WHERE weatherType LIKE "r%"`
    - Selects all records that include in weatherType a word that starts with "r".`
- The underscore _ wildcard matches any single character.
  + "_n" mathces "an", "bn", ...


## Example 4
Generate daily weather descriptions based on table “weatherType” for each location ID in table “metoffice_dailyweatherdata”. The descriptions should be generated as:
- When the weather type is exactly 'Clear night’, 'Sunny day’ or contains '%partly cloudy%’, the weather should be described as `OK Weather`;
- When the weather type contains 'heavy’ or 'snow’, the weather should be
described as `Bad weather`;
- When the weather type contains ‘light’ or ‘Drizzle’, the weather should be
described as `Not that great weather`;
- All the other weathers should be described as `No description`

For each location, there should be how many hours last under each
description.
### Hint
- Tables: metoffice_dailyweatherdata, weatherType
- Information:
  + location ID --- “locationID” from “metoffice_dailyweatherdata”;
  + New column “weather_description” (need to generate based on the weatherType table using conditional statement with `LIKE` operator);
  + New column “hours” (need to compute how many hours for each weather --- count the number of “obs_time” from table metoffice_dailyweatherdata )
- Relationship between tables:
  + “weatherTypeID”, `weatherType` in table `metoffice_dailyweatherdata`
  + `weatherTypeID` in table `weatherType`
- Conditions in WHERE: No
- Any Groups?
  + locationID
  + weather_description
  + for each location counting the number of “hours” `metoffice_dailyweatherdata.obs_time` for each unique (distinct) weather description.

In [ ]:
%%sql
SELECT daily.locationID,
  CASE
    WHEN wt.weatherType IN ('Clear night', 'Sunny day')
      OR wt.weatherType LIKE '%partly cloudy%'
      THEN 'OK Weather'
    WHEN wt.weatherType LIKE '%heavy%'
      OR wt.weatherType LIKE '%snow%'
      THEN 'Bad weather'
    WHEN wt.weatherType LIKE '%light%'
      OR wt.weatherType LIKE '%Drizzle%'
      THEN 'Not that great weather'
    ELSE 'No description'
  END
  AS weather_description,
  COUNT(DISTINCT daily.obs_time) AS hours
  FROM metoffice_dailyweatherdata daily
  LEFT JOIN weatherType wt
  ON daily.weatherType = wt.weatherTypeID
  GROUP BY
  daily.locationID, weather_description;

 * sqlite:///weather.db
Done.


LocationId,weather_description,hours
3002,Bad weather,6
3002,No description,24
3002,Not that great weather,4
3002,OK Weather,2
3005,Bad weather,1
3005,No description,24
3005,Not that great weather,10
3005,OK Weather,5
3008,Bad weather,4
3008,No description,24


## Views
A view is a virtual table that appears in the database like any other normal
table
- However a view is essentially a saved `SELECT` statement
- We say it’s a virtual table as it doesn’t take up any space.
- A `SELECT *` query on the view will basically trigger the saved SQL SELECT
statement behind the curtain
- We mainly use views for security purposes.
  + This is when we don’t want to give someone access to the whole table
(or multiple tables) and we just give them access to a View.
  + Another reason is when we want to explore the output of a complex SQL
query. In that case we may choose to create a view of that query and then
query that view instead.
- To create a view:
```sql
CREATE VIEW <view_name> AS SELECT...
```
- To drop a view:
```sql
DROP VIEW <view_name>
```

## Example 5
Create `windy_records` view that shows the number of windy records `(windspeed >= 50)` per day ordered by the number of windy records. Display and then drop the view.
### Hint
- Tables: metoffice_dailyweatherdata;
- Information:
  + `obs_date`
  + `number_of_windy_records`, use `SUM` to sum the values of `IFF` on windspeed to construct it.
- Conditions: No
- Any Groups?
  + obs_date
- Descending order by number of records.

In [ ]:
%%sql
CREATE VIEW windy_records AS
  SELECT
    obs_date,
    SUM(IIF (windspeed >= 50, 1, 0) )
      AS number_of_windy_records
  FROM metoffice_dailyweatherdata daily
  GROUP BY obs_date;
SELECT * FROM windy_records ORDER BY number_of_windy_records DESC;

 * sqlite:///weather.db
Done.
Done.


obs_date,number_of_windy_records
2020-01-02,40
2020-01-01,17
2020-01-03,17
2020-01-04,6


In [ ]:
%%sql
DROP VIEW windy_records;

 * sqlite:///weather.db
Done.


[]

# Exercises
MySQL can open multiple databases so we need to specify the name of the database (e.g. `DATABASE_NAME.TABLE_NAME`).
But `SQLite` (the SQL database that we use in this notebook) opens only one database at a time and we don't use the name of the database. As an example the table `cat_regions` is:
- **MySQL**: `weather_db.cat_regions`
- **SQLite**: `cat_regions`

## Exercise 1
Using tables `metoffice_dailyweatherdata` and `cat_locations` create a view called `carlisle_max_temperature` that contains the max temperature for Carlisle per time of day (morning `(time <12)`, afternoon `(time>=12 and time <18)`, evening `(time >=18)`).
The resulting view should contain two columns: `time_of_day` (morning, afternoon, evening), and `max_tempreture`.

###Hint
- Tables: “metoffice_dailyweatherdata” and “cat_locations”
- Information:
  + “time_of_day” (morning, afternoon, evening) --- Case/WHEN of IIF
  + “max_tempreture” --- MAX();
  + to get the hour as string for a given time
    - `strftime ('%H',obs_time)`
  + to compare with a number we need to cast it (e.g. convert to integer):
    - `CAST( strftime ('%H',obs_time) as INT)`
- Relationship between tables: location ID
- Conditions in WHERE: 'Carlisle'
- Any Groups? Yes, “time_of_day”.

In [21]:
%%sql
SELECT * FROM cat_locations WHERE Location = 'Carlisle'

 * sqlite:///weather.db
Done.


LocationID,Location,PostCode,Country,Region,region_description,Latitude,Longitude,Continent,Elevation,nationalPark,unitaryAuthArea
3220,Carlisle,None,None,nw,North West,54.933,-2.963,Europe,28.0,,Cumbria
350766,Carlisle,None,None,nw,North West,54.8924,-2.9315,Europe,21.0,,Cumbria


In [12]:
%%sql
SELECT * FROM metoffice_dailyweatherdata l

 * sqlite:///weather.db
Done.


LocationId,obs_dateTime,obs_date,obs_time,temperature,windspeed,humidity,dewpoint,pressure,windgust,visibility,winddirection,pressuretendency,timestamp,rainy,windy,snow,weatherType
3002,2020-01-01 00:00:00,2020-01-01,00:00:00,7.5,21,84.0,5.0,1018,32,13000,NULL,F,2020-01-01 06:00:03,0,0,0,8
3002,2020-01-01 01:00:00,2020-01-01,01:00:00,7.5,22,81.7,4.6,1018,34,12000,NULL,F,2020-01-01 06:00:03,0,0,0,8
3002,2020-01-01 02:00:00,2020-01-01,02:00:00,7.9,24,79.9,4.7,1017,36,11000,NULL,F,2020-01-01 06:00:03,0,0,0,8
3002,2020-01-01 03:00:00,2020-01-01,03:00:00,7.5,23,82.3,4.7,1016,40,13000,NULL,F,2020-01-01 06:00:03,0,0,0,8
3002,2020-01-01 04:00:00,2020-01-01,04:00:00,8.0,18,84.6,5.6,1015,33,12000,NULL,F,2020-01-01 06:00:03,0,0,0,8


In [22]:
%%sql
CREATE VIEW carlisle_max_temprature AS
SELECT
CASE
WHEN CAST(strftime('%H , obs_time') AS INT)<12 THEN 'MORNING'
WHEN CAST(strftome('%H , obs_tome') AS INT)>=18 THEN 'EVENING'
ELSE 'AFTERNONE'
END
AS time_of_day,
MAX(temperature) AS max_temprature
FROM metoffice_dailyweatherdata as daily
INNER JOIN
cat_locations as loc
ON daily.locationID = loc.locationID
WHERE loc.Location = 'Carlisle'
GROUP BY time_of_day

 * sqlite:///weather.db
Done.


[]

## Exercise 2
Based on table `metoffice_dailyweatherdata`, create a view called
`high_pressure_percentages` that returns how many cases on average we observed
pressure above 1020 per day, i.e. a percentage of cases that are classified as  above 1020.

Your resulting view should contain `obs_date` (observation date), and
`avg_high_pressure`. Note that you ou should work on those pressures which are not null.
### Hint
- Tables: “metoffice_dailyweatherdata”
- Information:
  + “obs_date” directly from “metoffice_dailyweatherdata”
  + “avg_high_pressure” --- AVG(IF or case when);
- Relationship between tables: no
- Conditions in WHERE: pressure IS NOT NULL
- Any Groups? Yes, “obs_date”.


In [23]:
%%sql
CREATE VIEW high_pressure_persentages AS
SELECT
obs_date,
AVG(IIF(pressure > 1020 , 1 , 0)) AS avg_high_pressure
FROM metoffice_dailyweatherdata
WHERE Pressure IS NOT NULL
GROUP BY obs_date;

 * sqlite:///weather.db
Done.


[]

## Exercise 3
Create a view called `rainy_snowy` that for each station returns a `0/1` flag if there was at least one rainy hour gives `1` otherwise `0`; and one similar `0/1` flag if there was at least one snowy hour gives `1` otherwise `0`.

You need to use table `metoffice_dailyweatherdata` and `weatherType`. The
resulting view should contain three columns: `locationID`,
`at_least_one_rainy_hour`, and `at_least_one_snowy_hour`.

### Hint
- Tables: “metoffice_dailyweatherdata” and “weatherType”
- Information:
  + You can use the `LIKE` statement to match `%rain%` or `%snow%` weather.
  + “locationID” directly from “metoffice_dailyweatherdata”
  + “at_least_one_rainy_hour” --- MAX(), IIF or CASE WHEN, LIKE
  + “at_least_one_snowy_hour” --- MAX(), IIF or CASE WHEN, LIKE
- Relationship between tables:
  + `metoffice_dailyweatherdata.weatherType = weatherType.weatherTypeID`
- Conditions in WHERE: no
- Any Groups?
  + Yes, “LocationID” from “metoffice_dailyweatherdata”.

In [53]:
%%sql
CREATE VIEW rainy_snowy AS
  SELECT
    MAX(IIF(wt.weatherType Like '%rain%' , 1 , 0)) AS at_least_one_rainy_hour,
    MAX(IIF(wt.weatherType Like '%snow%' , 1 , 0)) AS at_least_one_snowy_hour,
    daily.locationID
FROM
    metoffice_dailyweatherdata as daily
INNER JOIN
    weatherType as wt ON daily.weatherType = wt.weatherTypeID
GROUP BY daily.LocationID;
SELECT * from rainy_snowy limit 10;

 * sqlite:///weather.db
Done.
Done.


at_least_one_rainy_hour,at_least_one_snowy_hour,LocationId
1,0,3002
1,1,3005
1,1,3008
1,0,3017
1,0,3023
1,0,3026
1,0,3031
1,0,3034
0,0,3039
1,0,3044


In [52]:
%%sql
DROP VIEW rainy_snowy;

 * sqlite:///weather.db
Done.


[]

## Exercice 4
Create a view called `snow_weather_station_counts` which counts the number of
weather stations that had at least one snowy hour vs none. You need to work on the view created in exercise 3. The resulting view should contain two columns
`snow_counts` and `number_of_locations`.

### Hint
- Tables: “rainy_snowy” (view)
- Information:
  + “snow_counts” --- IIF() or CASE WHEN
  + “number_of_locations” --- COUNT(), DISTINCT()
- Relationship between tables: no
- Conditions in WHERE: no
- Any Groups?
  + Yes, “snow_counts”

In [33]:
%%sql
CREATE VIEW snow_weather_station_counts AS
SELECT
CASE
WHEN at_least_one_snowy_hour = 0 THEN 'no_snow_hour'
ELSE 'at_least_one_snow_hour'
END
AS snow_counts,
COUNT(DISTINCT LocationID) AS Number_of_Locations
FROM rainy_snowy
GROUP BY snow_counts


 * sqlite:///weather.db
Done.


[]

In [58]:
%%sql
SELECT * from snow_weather_station_counts limit 10;

 * sqlite:///weather.db
Done.


snow_counts,Number_of_Locations
at_least_one_snow_hour,2
no_snow_hour,138


## Exercise 5
Create a view known as `rain_or_snow_weather_station` that can:
- Give the weather categories based on weather rainy or snowy.
  + If there is **no snowy or rainy hours** the weather category should be set as `"No rain or snow"`.
  + **Otherwise**, the weather category should be set as `"Either rain or snow"`.
- For each weather category, count the number of weather stations.
Please working on the view obtained from exercise 3. The resulting view should
contain two columns: “weather_categorisation” and “number_of_stations”.

### Hint
- Tables: “rainy_snowy” (view)
- Information:
  + “weather_categorisation” --- IIF() or CASE WHEN
  + “number_of_stations” --- COUNT(), DISTINCT()
- Relationship between tables: no
- Conditions in WHERE: no
- Any Groups?
  + Yes, “weather_categorisation”.

In [57]:
%%sql
CREATE VIEW rain_or_snow_weather_station AS
SELECT
CASE
WHEN at_least_one_rainy_hour + at_least_one_snowy_hour = 0 THEN 'NO rain or snow'
ELSE 'Either rain or snow'
END
AS weather_categorisation,
COUNT(DISTINCT locationID) AS Number_of_stations
FROM rainy_snowy
GROUP BY
weather_categorisation;
SELECT * from rain_or_snow_weather_station limit 10;

 * sqlite:///weather.db
Done.
Done.


weather_categorisation,Number_of_stations
Either rain or snow,95
NO rain or snow,45
